In [1]:
import numpy as np
import pandas as pd
from scipy.stats import skew
import gc 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mode
from functools import reduce
import warnings 
warnings.filterwarnings("ignore", category=Warning)

In [2]:
data = pd.read_csv("../data/2_subject_with_48hrs_obs.csv")
data

,subject_id,hadm_id,stay_id,charttime,itemid,value,valuenum,valueuom,valuenum_capped,first_careunit,last_careunit,intime,outtime,los,duration_hours
0,10000690,25860671,37081114,2150-11-04 12:00:00,223835,50,50.00,NaN,50.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
1,10000690,25860671,37081114,2150-11-04 18:00:00,223835,70,70.00,NaN,70.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
2,10000690,25860671,37081114,2150-11-02 19:54:00,220045,79,79.00,bpm,79.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
3,10000690,25860671,37081114,2150-11-02 19:54:00,220179,107,107.00,mmHg,107.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
4,10000690,25860671,37081114,2150-11-02 19:54:00,220210,23,23.00,insp/min,23.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,3.893252,93.438056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8610466,19999987,23865745,36195440,2145-11-04 05:01:00,220645,147,147.00,mEq/L,147.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333
8610467,19999987,23865745,36195440,2145-11-04 05:01:00,225624,22,22.00,mg/dL,22.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333
8610468,19999987,23865745,36195440,2145-11-04 07:18:00,220224,69,69.00,mmHg,69.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333
8610469,19999987,23865745,36195440,2145-11-04 07:18:00,223830,7.42,7.42,units,7.42,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,1.937847,46.508333


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8610471 entries, 0 to 8610470
Data columns (total 15 columns):
 #   Column           Dtype  
---  ------           -----  
 0   subject_id       int64  
 1   hadm_id          int64  
 2   stay_id          int64  
 3   charttime        object 
 4   itemid           int64  
 5   value            object 
 6   valuenum         float64
 7   valueuom         object 
 8   valuenum_capped  float64
 9   first_careunit   object 
 10  last_careunit    object 
 11  intime           object 
 12  outtime          object 
 13  los              float64
 14  duration_hours   float64
dtypes: float64(4), int64(4), object(7)
memory usage: 985.4+ MB


In [4]:
data_to_be_agg = data.drop(['valuenum','value','valueuom','los','duration_hours'], axis=1).copy()
data_to_be_agg.head()

,subject_id,hadm_id,stay_id,charttime,itemid,valuenum_capped,first_careunit,last_careunit,intime,outtime
0,10000690,25860671,37081114,2150-11-04 12:00:00,223835,50.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17
1,10000690,25860671,37081114,2150-11-04 18:00:00,223835,70.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17
2,10000690,25860671,37081114,2150-11-02 19:54:00,220045,79.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17
3,10000690,25860671,37081114,2150-11-02 19:54:00,220179,107.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17
4,10000690,25860671,37081114,2150-11-02 19:54:00,220210,23.0,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17


In [5]:
data_to_be_agg["charttime"] = pd.to_datetime(data_to_be_agg["charttime"])
data_to_be_agg["intime"] = pd.to_datetime(data_to_be_agg["intime"])

data_to_be_agg['hours_from_intime'] = (data_to_be_agg['charttime'] - data_to_be_agg['intime']) / pd.Timedelta(hours=1)
data_to_be_agg

,subject_id,hadm_id,stay_id,charttime,itemid,valuenum_capped,first_careunit,last_careunit,intime,outtime,hours_from_intime
0,10000690,25860671,37081114,2150-11-04 12:00:00,223835,50.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,40.383333
1,10000690,25860671,37081114,2150-11-04 18:00:00,223835,70.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,46.383333
2,10000690,25860671,37081114,2150-11-02 19:54:00,220045,79.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,0.283333
3,10000690,25860671,37081114,2150-11-02 19:54:00,220179,107.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,0.283333
4,10000690,25860671,37081114,2150-11-02 19:54:00,220210,23.00,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2150-11-02 19:37:00,2150-11-06 17:03:17,0.283333
...,...,...,...,...,...,...,...,...,...,...,...
8610466,19999987,23865745,36195440,2145-11-04 05:01:00,220645,147.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,30.033333
8610467,19999987,23865745,36195440,2145-11-04 05:01:00,225624,22.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,30.033333
8610468,19999987,23865745,36195440,2145-11-04 07:18:00,220224,69.00,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,32.316667
8610469,19999987,23865745,36195440,2145-11-04 07:18:00,223830,7.42,Trauma SICU (TSICU),Trauma SICU (TSICU),2145-11-02 22:59:00,2145-11-04 21:29:30,32.316667


In [6]:
time_windows = {
    'full': (0, 48),
    'first_10': (0, 4.8),
    'first_25': (0, 12.0),
    'first_50': (0, 24.0),
    'last_50': (24.0, 48.0),
    'last_25': (36.0, 48.0),
    'last_10': (43.2, 48.0)}

In [16]:
GCS_items = {220739, 223900, 223901}

def extract_item_features(df, itemid):
    df_item = df[df['itemid'] == itemid]
    if df_item.empty:
        return pd.DataFrame() 

    feature_col_rows = []
    for subject_id, group in df_item.groupby('subject_id'):
        features_cols = {}
        for window_name, (start_hr, end_hr) in time_windows.items():
            window_data = group[(group['hours_from_intime'] >= start_hr) & (group['hours_from_intime'] <= end_hr)]
            values = window_data['valuenum_capped'].dropna().values
            prefix = f'item{itemid}_{window_name}'
            
            features_cols[f'{prefix}_missing'] = int(len(values) == 0)
            
            if len(values) > 0:
                if itemid in GCS_items:
                    features_cols[f'{prefix}_min'] = np.min(values)
                    features_cols[f'{prefix}_max'] = np.max(values)
                    mode_val, count = mode(values, keepdims=True)
                    features_cols[f'{prefix}_mode'] = mode_val[0] if mode_val.size > 0 else np.nan
                    features_cols[f'{prefix}_count'] = len(values)

                else:
                    features_cols[f'{prefix}_min'] = np.min(values)
                    features_cols[f'{prefix}_max'] = np.max(values)
                    features_cols[f'{prefix}_mean'] = np.mean(values)
                    features_cols[f'{prefix}_std'] = np.std(values, ddof=1) if len(values) > 1 else 0.0
                    if len(values) > 2 and np.std(values) > 1e-6:
                        features_cols[f'{prefix}_skew'] = skew(values)
                    else:
                        features_cols[f'{prefix}_skew'] = 0.0
                    features_cols[f'{prefix}_count'] = len(values)
            else:
                features_cols[f'{prefix}_min'] = np.nan
                features_cols[f'{prefix}_max'] = np.nan
                features_cols[f'{prefix}_mean'] = np.nan
                features_cols[f'{prefix}_std'] = np.nan
                features_cols[f'{prefix}_skew'] = np.nan
                features_cols[f'{prefix}_count'] = 0

                if itemid in GCS_items:
                    features_cols[f'{prefix}_mode'] = np.nan
        
        features_cols['hadm_id'] = group['hadm_id'].iloc[0]
        features_cols['subject_id'] = subject_id
        feature_col_rows.append(pd.Series(features_cols))
    
    gc.collect()
    return pd.DataFrame(feature_col_rows)

In [8]:
itemid_name_map = {
    220045: 'HeartRate',
    220739: 'GCS_Eye',
    223900: 'GCS_Verbal',
    223901: 'GCS_Motor',
    220210: 'RespiratoryRate',
    220645: 'Sodium',
    220615: 'Creatinine',
    225624: 'BUN',
    220545: 'Hematocrit',
    223761: 'TempF',
    220179: 'NIBP_Systolic',
    223835: 'FiO2',
    223830: 'pH_Arterial',
    220224: 'PaO2'}

def rename_item_columns(df, itemid):
    readable_name = itemid_name_map.get(itemid, f'item{itemid}')
    rename_map = {}
    for col in df.columns:
        if f'item{itemid}_' in col:
            new_col = col.replace(f'item{itemid}_', f'{readable_name}_')
            rename_map[col] = new_col
    return df.rename(columns=rename_map)

In [9]:
features_1 = []

feature_to_compute=[220645, 220615, 225624]

for itemid in feature_to_compute:
    df = extract_item_features(data_to_be_agg, itemid)
    df = rename_item_columns(df, itemid)
    features_1.append(df)

In [10]:
features_2 = []

feature_to_compute=[220045, 220210, 220545]

for itemid in feature_to_compute:
    df = extract_item_features(data_to_be_agg, itemid)
    df = rename_item_columns(df, itemid)
    features_2.append(df)

In [11]:
features_3 = []

feature_to_compute=[223835, 223830, 220224]

for itemid in feature_to_compute:
    df = extract_item_features(data_to_be_agg, itemid)
    df = rename_item_columns(df, itemid)
    features_3.append(df)

In [12]:
features_4 = []

feature_to_compute=[223761, 220179]

for itemid in feature_to_compute:
    df = extract_item_features(data_to_be_agg, itemid)
    df = rename_item_columns(df, itemid)
    features_4.append(df)

In [17]:
features_5= []

feature_to_compute=[220739, 223900, 223901]

for itemid in feature_to_compute:
    df = extract_item_features(data_to_be_agg, itemid)
    df = rename_item_columns(df, itemid)
    features_5.append(df)

In [18]:
agg_feature = []
for block in [features_1, features_2, features_3, features_4, features_5]:
    agg_feature.extend(block)

In [19]:
all_agg_features = reduce(
    lambda left, right: pd.merge(left, right, on= ['subject_id','hadm_id'], how='outer'), agg_feature)
all_agg_features.head()

,Sodium_full_missing,Sodium_full_min,Sodium_full_max,Sodium_full_mean,Sodium_full_std,Sodium_full_skew,Sodium_full_count,Sodium_first_10_missing,Sodium_first_10_min,Sodium_first_10_max,...,GCS_Motor_first_10_skew,GCS_Motor_first_25_mean,GCS_Motor_first_25_std,GCS_Motor_first_25_skew,GCS_Motor_first_50_mean,GCS_Motor_first_50_std,GCS_Motor_first_50_skew,GCS_Motor_full_mean,GCS_Motor_full_std,GCS_Motor_full_skew
0,0.0,133.0,137.0,134.666667,2.081666,0.528005,3.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,138.0,138.0,138.000000,0.000000,0.000000,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,138.0,140.0,139.000000,1.414214,0.000000,2.0,0.0,140.0,140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.0,136.0,140.0,138.333333,2.081666,-0.528005,3.0,0.0,140.0,140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,125.0,134.0,129.833333,3.188521,-0.240304,6.0,0.0,125.0,125.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
gcs_eye_cols = [col for col in all_agg_features.columns 
                if col.startswith('GCS') and 
                   any(stat in col for stat in ['mean', 'std', 'skew'])]

all_agg_features = all_agg_features.drop(columns=gcs_eye_cols)

In [21]:
all_agg_features

,Sodium_full_missing,Sodium_full_min,Sodium_full_max,Sodium_full_mean,Sodium_full_std,Sodium_full_skew,Sodium_full_count,Sodium_first_10_missing,Sodium_first_10_min,Sodium_first_10_max,...,GCS_Motor_last_25_missing,GCS_Motor_last_25_min,GCS_Motor_last_25_max,GCS_Motor_last_25_mode,GCS_Motor_last_25_count,GCS_Motor_last_10_missing,GCS_Motor_last_10_min,GCS_Motor_last_10_max,GCS_Motor_last_10_mode,GCS_Motor_last_10_count
0,0.0,133.0,137.0,134.666667,2.081666,0.528005,3.0,1.0,NaN,NaN,...,0.0,6.0,6.0,6.0,3.0,0.0,6.0,6.0,6.0,1.0
1,0.0,138.0,138.0,138.000000,0.000000,0.000000,1.0,1.0,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
2,0.0,138.0,140.0,139.000000,1.414214,0.000000,2.0,0.0,140.0,140.0,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
3,0.0,136.0,140.0,138.333333,2.081666,-0.528005,3.0,0.0,140.0,140.0,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
4,0.0,125.0,134.0,129.833333,3.188521,-0.240304,6.0,0.0,125.0,125.0,...,0.0,6.0,6.0,6.0,1.0,0.0,6.0,6.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
44417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
44418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
44419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0


In [22]:
# Reorder columns after all merges
cols = all_agg_features.columns.tolist()

# Move 'subject_id' and 'hadm_id' to the front
ordered_cols = ['subject_id', 'hadm_id'] + [col for col in cols if col not in ['subject_id', 'hadm_id']]
all_agg_features = all_agg_features[ordered_cols]
all_agg_features

,subject_id,hadm_id,Sodium_full_missing,Sodium_full_min,Sodium_full_max,Sodium_full_mean,Sodium_full_std,Sodium_full_skew,Sodium_full_count,Sodium_first_10_missing,...,GCS_Motor_last_25_missing,GCS_Motor_last_25_min,GCS_Motor_last_25_max,GCS_Motor_last_25_mode,GCS_Motor_last_25_count,GCS_Motor_last_10_missing,GCS_Motor_last_10_min,GCS_Motor_last_10_max,GCS_Motor_last_10_mode,GCS_Motor_last_10_count
0,10000690.0,25860671.0,0.0,133.0,137.0,134.666667,2.081666,0.528005,3.0,1.0,...,0.0,6.0,6.0,6.0,3.0,0.0,6.0,6.0,6.0,1.0
1,10001217.0,24597018.0,0.0,138.0,138.0,138.000000,0.000000,0.000000,1.0,1.0,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
2,10001725.0,25563031.0,0.0,138.0,140.0,139.000000,1.414214,0.000000,2.0,0.0,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
3,10002013.0,23581541.0,0.0,136.0,140.0,138.333333,2.081666,-0.528005,3.0,0.0,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
4,10002114.0,27793700.0,0.0,125.0,134.0,129.833333,3.188521,-0.240304,6.0,0.0,...,0.0,6.0,6.0,6.0,1.0,0.0,6.0,6.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44416,19830515.0,24157504.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
44417,19843180.0,25449392.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
44418,19871603.0,22686287.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
44419,19954126.0,27026518.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0


In [23]:
all_agg_features['subject_id'] = all_agg_features['subject_id'].astype('Int64')
all_agg_features['hadm_id'] = all_agg_features['hadm_id'].astype('Int64')
all_agg_features

,subject_id,hadm_id,Sodium_full_missing,Sodium_full_min,Sodium_full_max,Sodium_full_mean,Sodium_full_std,Sodium_full_skew,Sodium_full_count,Sodium_first_10_missing,...,GCS_Motor_last_25_missing,GCS_Motor_last_25_min,GCS_Motor_last_25_max,GCS_Motor_last_25_mode,GCS_Motor_last_25_count,GCS_Motor_last_10_missing,GCS_Motor_last_10_min,GCS_Motor_last_10_max,GCS_Motor_last_10_mode,GCS_Motor_last_10_count
0,10000690,25860671,0.0,133.0,137.0,134.666667,2.081666,0.528005,3.0,1.0,...,0.0,6.0,6.0,6.0,3.0,0.0,6.0,6.0,6.0,1.0
1,10001217,24597018,0.0,138.0,138.0,138.000000,0.000000,0.000000,1.0,1.0,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
2,10001725,25563031,0.0,138.0,140.0,139.000000,1.414214,0.000000,2.0,0.0,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
3,10002013,23581541,0.0,136.0,140.0,138.333333,2.081666,-0.528005,3.0,0.0,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
4,10002114,27793700,0.0,125.0,134.0,129.833333,3.188521,-0.240304,6.0,0.0,...,0.0,6.0,6.0,6.0,1.0,0.0,6.0,6.0,6.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44416,19830515,24157504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
44417,19843180,25449392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
44418,19871603,22686287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0
44419,19954126,27026518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0


In [24]:
all_agg_features.to_csv('../data/4_feature_aggregated.csv', index=False)